# Tutorial for setting up and simulating dsDNA with MRG-CG DNA model

In [ ]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import mdtraj

try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import MRGdsDNAParser
from openabc.forcefields import MOFFMRGModel

We simulate a 200 bp dsDNA by MRG-CG DNA model. Similar to MOFF, for dsDNA, we also provide a simple way to parse DNA topology and get all the bonds, angles, fan bonds, etc. Use class `MRGdsDNAParser` to parse each dsDNA. The nonbonded exclusion list includes CG DNA atoms involved in bonds or angles, but those involved in fan bonds are not included. It is important that the parser cannot automatically recognize which nucleotides should be paired, so the input has to be the atomistic model of a single dsDNA with correct nucleotide index so that nucleotide i is paired with nucleotide N - i - 1 (nucleotide index starts from 0 and N is the total number of nucleotides in dsDNA). 

Also, as the original MRG-CG DNA model is designed with explicit ions, to apply this model with implicit ion, by default we scale all the bonded interaction (bond, angle, and fan bond) force constants by 0.9 to keep correct thermodynamic properties such as the persistence length. 

In [ ]:
# parse dsDNA with atomistic model as input
dsDNA = MRGdsDNAParser.from_atomistic_pdb('input-pdb/all_atom_200bpDNA.pdb', 'MRG_dsDNA.pdb')

Let's take a look at the bonds, angles, and fan bonds. You can see the force constants are scaled. 

In [ ]:
print(dsDNA.dna_bonds.head())
print(dsDNA.dna_angles.head())
print(dsDNA.dna_fan_bonds.head())

Now we can build the system and run simulation. We first build an instance of `MOFFMRGModel`, then we append the dsDNA parser instance into it and set up the simulation. 

In [ ]:
dna = MOFFMRGModel()
dna.append_mol(dsDNA)
top = app.PDBFile('MRG_dsDNA.pdb').getTopology()
dna.create_system(top, box_a=100, box_b=100, box_c=100)
salt_conc = 100*unit.millimolar
temperature = 300*unit.kelvin
dna.add_dna_bonds(force_group=1)
dna.add_dna_angles(force_group=2)
dna.add_dna_fan_bonds(force_group=3)
dna.add_contacts(force_group=4)
dna.add_elec_switch(salt_conc, temperature, force_group=5) # electrostatic interaction depends on salt concentration and temperature
dna.save_system('dsDNA.xml')
collision = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.NoseHooverIntegrator(temperature, collision, timestep)
platform_name = 'CPU'
init_coord = app.PDBFile('MRG_dsDNA.pdb').getPositions()
dna.set_simulation(integrator, platform_name, init_coord=init_coord)
dna.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output.dcd'
dna.add_reporters(output_interval, output_dcd)
dna.simulation.context.setVelocitiesToTemperature(temperature)
dna.simulation.step(500)


In [ ]:
# view trajectory
traj = mdtraj.load_dcd('output.dcd', top='MRG_dsDNA.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view